# Blog Website with Flask
This Jupyter Notebook contains a complete implementation of a simple blog website using Flask with user authentication and content management.

In [25]:

# Install necessary packages (only once)
!pip install flask flask-login flask_sqlalchemy


In [29]:

from flask import Flask, render_template_string, redirect, url_for, request, flash
from flask_sqlalchemy import SQLAlchemy
from flask_login import LoginManager, UserMixin, login_user, login_required, logout_user, current_user
from werkzeug.security import generate_password_hash, check_password_hash
import os
from threading import Thread


In [31]:

app = Flask(__name__)
app.config['SECRET_KEY'] = 'mysecretkey'
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///blog.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False

db = SQLAlchemy(app)
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'


In [33]:

class User(UserMixin, db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(150), unique=True, nullable=False)
    password = db.Column(db.String(150), nullable=False)

class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    title = db.Column(db.String(100))
    content = db.Column(db.Text)
    author_id = db.Column(db.Integer, db.ForeignKey('user.id'))


In [35]:

@login_manager.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))


In [37]:

if 'index' not in app.view_functions:
    @app.route('/')
    def index():
        posts = Post.query.all()
        return render_template_string("""
        <h1>Welcome to the Blog</h1>
        {% if current_user.is_authenticated %}
        <p>Hello, {{ current_user.username }}! <a href='/logout'>Logout</a></p>
        <p><a href='/dashboard'>Dashboard</a></p>
        {% else %}
        <a href='/login'>Login</a> | <a href='/register'>Register</a>
        {% endif %}
        <hr>
        {% for post in posts %}
        <h2>{{ post.title }}</h2><p>{{ post.content }}</p><hr>
        {% endfor %}
        """, posts=posts)


In [39]:

if 'register' not in app.view_functions:
    @app.route('/register', methods=['GET', 'POST'])
    def register():
        if request.method == 'POST':
            username = request.form['username']
            password = request.form['password']
            hashed_pw = generate_password_hash(password)
            new_user = User(username=username, password=hashed_pw)
            db.session.add(new_user)
            db.session.commit()
            return redirect(url_for('login'))
        return render_template_string("""
        <h2>Register</h2>
        <form method='POST'>
            Username: <input name='username'><br>
            Password: <input name='password' type='password'><br>
            <input type='submit'>
        </form>
        """)


In [41]:

if 'login' not in app.view_functions:
    @app.route('/login', methods=['GET', 'POST'])
    def login():
        if request.method == 'POST':
            user = User.query.filter_by(username=request.form['username']).first()
            if user and check_password_hash(user.password, request.form['password']):
                login_user(user)
                return redirect(url_for('index'))
            flash('Invalid credentials')
        return render_template_string("""
        <h2>Login</h2>
        <form method='POST'>
            Username: <input name='username'><br>
            Password: <input name='password' type='password'><br>
            <input type='submit'>
        </form>
        """)


In [43]:

if 'logout' not in app.view_functions:
    @app.route('/logout')
    @login_required
    def logout():
        logout_user()
        return redirect(url_for('index'))


In [45]:

if 'dashboard' not in app.view_functions:
    @app.route('/dashboard', methods=['GET', 'POST'])
    @login_required
    def dashboard():
        if request.method == 'POST':
            title = request.form['title']
            content = request.form['content']
            new_post = Post(title=title, content=content, author_id=current_user.id)
            db.session.add(new_post)
            db.session.commit()
        return render_template_string("""
        <h2>Dashboard</h2>
        <form method='POST'>
            Title: <input name='title'><br>
            Content:<br><textarea name='content'></textarea><br>
            <input type='submit'>
        </form><br>
        <a href='/'>Back to Home</a>
        """)


In [49]:

with app.app_context():
    db.create_all()

def run_app():
    app.run(debug=True, use_reloader=False)

Thread(target=run_app).start()


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
